In [1]:
import json
import math
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# I. Load the data & some insights

In [2]:
# We load the 3 datasets 
df_business = pd.read_csv('./data/df_business.csv')
df_reviews = pd.read_csv('./data/df_reviews.csv')
df_users = pd.read_csv('./data/df_user.csv')

In [3]:
# Convert the string of friends to a list
df_users['friend_list'] = df_users.apply(lambda x: x.friends.split(', '), axis=1)
df_user = df_users.drop('friends', axis=1)

For reading purposes, the 3 datasets have the following form

In [4]:
df_business.head(3)

,business_id,state,latitude,longitude,stars
0,f9NumwFMBDn751xgFiRbNA,NC,35.462724,-80.852612,3.5
1,Yzvjg0SayhoZgCljUJRF9Q,AZ,33.569404,-111.890264,5.0
2,XNoUzKckATkOD1hP6vghZg,QC,45.479984,-73.580070,5.0


In [5]:
df_reviews.head(3)

,review_id,user_id,business_id,stars,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,2015-12-05 03:18:11


In [6]:
df_users.head(3)

,user_id,friends,friend_list
0,ntlvfPzc8eglqvk92iDIAw,"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg...","[oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfD..."
1,FOBRPlBHa3WPHFB5qYDlVg,"ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOug...","[ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOu..."
2,zZUnPeh2hEp0WydbAZEOOg,"Uwlk0txjQBPw_JhHsQnyeg, Ybxr1tSCkv3lYA0I1qmnPQ...","[Uwlk0txjQBPw_JhHsQnyeg, Ybxr1tSCkv3lYA0I1qmnP..."


In [7]:
print('There are', df_business.shape[0], 'businesses')
print('There are', df_reviews.shape[0], 'reviews')
print('There are', df_users.shape[0], 'users')

There are 209393 businesses
There are 8021122 reviews
There are 1968703 users


For our study, we need to infer user's home, so we need **at least 3 reviews**. Also, we will study users with **at least 3 friends**.  

In [8]:
# We count the number of reviewe per user
df_numberOfReviews = df_reviews.groupby('user_id').count().drop(['business_id', 'stars', 'date'], axis = 1).rename(columns={'review_id': 'review_count'})

In [9]:
df_numberOfReviews.head()

,review_count
user_id,
---1lKK3aKOuomHnwAkAow,131
---3o4ZsKYoBYBe7H6xG8A,1
---89pEy_h9PvHwcHNbpyg,1
---94vtJ_5o_nikEs6hUjg,5
---PLwSf5gKdIoVnyRHgBA,3


In [10]:
# We merge the df_users dataset to the df_numberOfReviews dataset
df_users = pd.merge(df_users, df_numberOfReviews, left_on='user_id', right_on='user_id', how='inner')

In [11]:
# We create a mask for users with more than 3 friends
mask = df_users.apply(lambda x: len(x.friend_list) >= 3, axis=1)

In [12]:
print('Number of users with more than 3 reviews:', df_numberOfReviews[df_numberOfReviews['review_count'] >= 3].shape[0])
print('Number of users with more than 3 friends:', df_users[mask].shape[0])

Number of users with more than 3 reviews: 602736
Number of users with more than 3 friends: 957463


In [13]:
# Select users that satisfy both conditions
df_selectedUsers = df_users[mask&(df_users['review_count'] >= 3)]
print('Number of users with more than 4 reviews and with more than 3 friends:', df_selectedUsers.shape[0])

Number of users with more than 4 reviews and with more than 3 friends: 343424


In [69]:
friends_dict = df_selectedUsers.set_index("user_id")["friend_list"].to_dict()

In [14]:
# Number of checkins of the selected users
print('Number of checkins of selected users:', df_selectedUsers['review_count'].sum())

Number of checkins of selected users: 4516345


In [15]:
df_reviews = pd.merge(df_reviews, df_business, left_on='business_id', right_on='business_id', how='inner').drop(['state'], axis=1)

In [17]:
df_selectedReviews = df_reviews[df_reviews["user_id"].isin(df_selectedUsers.user_id)]

In [57]:
df_selectedReviews["date"] = pd.to_datetime(df_selectedReviews.date.values)

<ipython-input-57-f1ff465342bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selectedReviews["date"] = pd.to_datetime(df_selectedReviews.date.values)


In [204]:
df_selectedReviews

,review_id,user_id,business_id,stars_x,date,latitude,longitude,stars_y
1,t7xOZF5UKXjSpVcXLOSAgw,owbC7FP8SNAlwv6f9S5Stw,-MhfebM0QIsKt87iDN-FNw,2,2014-03-14 08:24:25,36.112896,-115.177637,3.5
2,MimB5Xh85rG7phUMPrShag,v9vGnjphb0Hta0lvtf5haA,-MhfebM0QIsKt87iDN-FNw,3,2015-10-07 22:16:59,36.112896,-115.177637,3.5
4,cnV5xtm6WuyaLfot9uWbDg,LkWNo83Lg92C5V4JEyxOZA,-MhfebM0QIsKt87iDN-FNw,3,2010-10-10 01:27:31,36.112896,-115.177637,3.5
5,i593z2rGxk5Lj23Bx9o1Lg,mLtaUzSjIFO_3BmfPNUKqg,-MhfebM0QIsKt87iDN-FNw,1,2015-08-20 01:10:53,36.112896,-115.177637,3.5
6,TWpbq_vbpJRotCQ6l8SeXA,wKX1tAeRIYPU4NtM-R5N2w,-MhfebM0QIsKt87iDN-FNw,5,2012-03-31 22:00:15,36.112896,-115.177637,3.5
...,...,...,...,...,...,...,...,...
8021103,aoc4p8TMwZPGArkWJwp_3A,MqeC_AaJ2sUp9F0xfm-kZw,p4yQ3QJbRTwjUS3dxP23EA,5,2019-06-17 19:44:32,33.678238,-112.101291,5.0
8021107,kPB6u7yU0p04jsaTM2kg4g,p0ndG4QEa0D502O9D07hmA,zf2-GEQ32-tN-sxUn3Te6A,5,2019-12-11 21:40:24,35.988448,-115.091844,5.0
8021109,SuwjID5CulwUpQrrpieaGA,PcfjPDJjlAbFTHyxDys8YA,zf2-GEQ32-tN-sxUn3Te6A,5,2019-07-24 01:50:00,35.988448,-115.091844,5.0
8021110,hLbvNMY8XLSs3r6NhxXapg,4xG1aexd2byltBd4rr-puw,_wEWbwthVLxc0fefm8Kqpg,5,2016-01-16 23:40:10,33.559234,-112.326905,5.0


# Getting the Reviewed Label

In [33]:
df_reviews["date"] = pd.to_datetime(df_reviews.date.values)

In [47]:
df_reviews = df_reviews.rename({"stars_x" : "rating", "stars_y" : "average_rating"},axis=1)

In [49]:
df_reviews_groupedUser = df_reviews.groupby(["user_id","business_id"])[["date","rating"]].agg(list)

In [139]:
df_reviews_groupedUser

date rating
user_id                business_id                                         
---1lKK3aKOuomHnwAkAow --9e1ONYQuAa-CB_Rrw7Tw  [2008-11-11 04:40:05]    [4]
                       -ErwgUmZ1-jHW_rSu55jlg  [2010-11-09 20:21:52]    [5]
                       0YhT9HCBkU394IG6tQVtNQ  [2012-05-01 16:17:54]    [1]
                       1JgaRBX0oiRsvEhHF3ZMjw  [2011-02-16 03:58:48]    [1]
                       1Vn_lex3LGGwuTo-xeJnww  [2011-02-16 04:16:12]    [5]
...                                                              ...    ...
zzzfcCWPDkg-kGVpqCKCEg WcHTXumqKMt0WdF-9RWljw  [2015-01-19 04:43:56]    [5]
zzzhFxwfzL56CQVnw8piTw wHwHmozyqtIE5U3uTtvpuA  [2016-06-20 19:29:23]    [5]
zzzmshdEWLFCApxETl1TGQ 74pWjFssgqyhaVg-YCUo3A  [2013-01-05 15:16:10]    [5]
                       WivFagb6JtTeJskEEvYfXg  [2017-01-31 16:00:51]    [5]
zzzv9W8ShM4TkXZGMFVY1w 6WU_JVpPPnbAfBvMcRoPuw  [2019-08-20 16:55:44]    [5]

[7735091 rows x 2 columns]

In [142]:
level_user_id = df_reviews_groupedUser.groupby(level='user_id')

In [156]:
user_business_date_dict = {}

for idx, data in level_user_id:
    user_business_date_dict[idx] = {}
    for row_index, row in data.iterrows():
        user_business_date_dict[idx][row_index[1]] = (row.date,row.rating)

In [188]:
user_business_date_dict.get('---1lKK3aKOuomHnwAkAow', {}).get('1JgaRBX0oiRsvEhHF3ZMjw')

([Timestamp('2011-02-16 03:58:48')], [1])

In [193]:
#takes a row of the shape ["user_id","business_id","date"]
#returns a label : if a friend has left a good review in the same place before : 2, if a friend has left a bad review
#in the same place before : 1, and if a friend hasn't left a review before : 0.
def get_visit_label(user_id,business_id,date_user):
    #get friend list of user:
    friends = friends_dict.get(user_id)
    for friend_id in friends:
        friend_dates_rating = user_business_date_dict.get(friend_id, {}).get(business_id)
        if(friend_dates_rating != None):
            for date,rating in zip(friend_dates_rating[0],friend_dates_rating[1]):
                if(date < date_user):
                    return rating
    return -1

In [190]:
selected_reviews_array = np.array(df_selectedReviews[["user_id","business_id","date"]])

In [194]:
get_visit_label_vec = np.vectorize(get_visit_label)

In [200]:
labels = get_visit_label_vec(selected_reviews_array[:,0],selected_reviews_array[:,1],selected_reviews_array[:,2])

In [205]:
df_selectedReviews["reviewed_by_friend_before"] = labels

<ipython-input-205-6ce838206260>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selectedReviews["reviewed_by_friend_before"] = labels


In [211]:
df_selectedReviews[df_selectedReviews["reviewed_by_friend_before"] > 3]

,review_id,user_id,business_id,stars_x,date,latitude,longitude,stars_y,reviewed_by_friend_before
6,TWpbq_vbpJRotCQ6l8SeXA,wKX1tAeRIYPU4NtM-R5N2w,-MhfebM0QIsKt87iDN-FNw,5,2012-03-31 22:00:15,36.112896,-115.177637,3.5,4
7,6dUXKtS2U59I8-NM04ExJQ,epkRB3J_oBEiyiwahvdmGg,-MhfebM0QIsKt87iDN-FNw,3,2013-05-09 18:07:55,36.112896,-115.177637,3.5,4
8,RaSU-d08Y2ywamgI3TOeRA,eZZyuJDouIg4p-GYB3PV_A,-MhfebM0QIsKt87iDN-FNw,2,2013-05-11 02:05:53,36.112896,-115.177637,3.5,4
11,k3aclaqIMx4lNLZ9D1Dp2Q,t_9U_94WuP_HUSEXnBgL1A,-MhfebM0QIsKt87iDN-FNw,3,2013-09-05 19:08:25,36.112896,-115.177637,3.5,4
14,JozE6cVCdizAqqzNRwvQcw,e1WA0YE3JpTg2Zb_lrHKQA,-MhfebM0QIsKt87iDN-FNw,5,2017-12-03 22:40:20,36.112896,-115.177637,3.5,4
...,...,...,...,...,...,...,...,...,...
8020163,Uo_FQ6zyE9l8k4o5UyCMuA,cnyCyr5az4YzlQ1oCSYNvQ,cZrtB-AWUd1BQSGchnTbIw,4,2019-03-07 22:47:09,36.239839,-115.123964,5.0,5
8020489,iF4WsvbKTfJ-cmHX855p6g,A6fUixxQqi5ecjCTboNKGQ,H5S3w4X8pik6Nef-JYXlbw,3,2015-08-18 22:06:30,33.633673,-111.949657,3.5,4
8020518,wLfLHmD947nzm9eqvIoLCw,va6mMvVp7k4-1RmSE0C8vw,M-aZPC-CQ9UhuzXpcLDH1A,5,2019-10-18 15:51:30,43.574056,-79.715096,5.0,5
8020543,mV44m0moZylmsQKYuCee-w,a749TTfV1cZQD6yow3_cgQ,tCd1NJH7T3rXvwDLuEtweQ,5,2019-09-04 05:01:08,36.116472,-115.261393,5.0,5


In [216]:
def set_label(rating):
    if rating < 0:
        return 0
    elif rating <4:
        return 1
    else :
        return 2

In [217]:
df_selectedReviews["reviewed_before_friend_4"] = df_selectedReviews["reviewed_by_friend_before"].apply(set_label)

<ipython-input-217-95bde75a12d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selectedReviews["reviewed_before_friend_4"] = df_selectedReviews["reviewed_by_friend_before"].apply(set_label)


In [218]:
df_selectedReviews

,review_id,user_id,business_id,stars_x,date,latitude,longitude,stars_y,reviewed_by_friend_before,reviewed_before_friend_4
1,t7xOZF5UKXjSpVcXLOSAgw,owbC7FP8SNAlwv6f9S5Stw,-MhfebM0QIsKt87iDN-FNw,2,2014-03-14 08:24:25,36.112896,-115.177637,3.5,-1,0
2,MimB5Xh85rG7phUMPrShag,v9vGnjphb0Hta0lvtf5haA,-MhfebM0QIsKt87iDN-FNw,3,2015-10-07 22:16:59,36.112896,-115.177637,3.5,-1,0
4,cnV5xtm6WuyaLfot9uWbDg,LkWNo83Lg92C5V4JEyxOZA,-MhfebM0QIsKt87iDN-FNw,3,2010-10-10 01:27:31,36.112896,-115.177637,3.5,2,1
5,i593z2rGxk5Lj23Bx9o1Lg,mLtaUzSjIFO_3BmfPNUKqg,-MhfebM0QIsKt87iDN-FNw,1,2015-08-20 01:10:53,36.112896,-115.177637,3.5,-1,0
6,TWpbq_vbpJRotCQ6l8SeXA,wKX1tAeRIYPU4NtM-R5N2w,-MhfebM0QIsKt87iDN-FNw,5,2012-03-31 22:00:15,36.112896,-115.177637,3.5,4,2
...,...,...,...,...,...,...,...,...,...,...
8021103,aoc4p8TMwZPGArkWJwp_3A,MqeC_AaJ2sUp9F0xfm-kZw,p4yQ3QJbRTwjUS3dxP23EA,5,2019-06-17 19:44:32,33.678238,-112.101291,5.0,-1,0
8021107,kPB6u7yU0p04jsaTM2kg4g,p0ndG4QEa0D502O9D07hmA,zf2-GEQ32-tN-sxUn3Te6A,5,2019-12-11 21:40:24,35.988448,-115.091844,5.0,-1,0
8021109,SuwjID5CulwUpQrrpieaGA,PcfjPDJjlAbFTHyxDys8YA,zf2-GEQ32-tN-sxUn3Te6A,5,2019-07-24 01:50:00,35.988448,-115.091844,5.0,-1,0
8021110,hLbvNMY8XLSs3r6NhxXapg,4xG1aexd2byltBd4rr-puw,_wEWbwthVLxc0fefm8Kqpg,5,2016-01-16 23:40:10,33.559234,-112.326905,5.0,-1,0
